In [18]:
### basic libraries
import os
from json import dumps

### carto libraries
import shapefile
import geopandas

from app.scripts.topojson import topojson

from pyproj import Proj, transform
inCRS  = 'epsg:2154'
outCRS = 'epsg:4326'
inProj  = Proj(init=inCRS) # proj in  : Lambert 93
outProj = Proj(init=outCRS) # proj out : WSG 84

### TO DO : find topojson libraries to reduce output file size


In [19]:
### basic folders addresses and names
cwd = os.getcwd()

data_folder = "app/static/data"
carto_folder = "carto"

carto_path = os.path.join(cwd, data_folder, carto_folder)

print "-- cwd :", cwd
print "-- carto path : ", carto_path


-- cwd : /Users/jpy/Dropbox/_FLASK/concours_pesticides
-- carto path :  /Users/jpy/Dropbox/_FLASK/concours_pesticides/app/static/data/carto


In [20]:
# cf : https://pypi.python.org/pypi/pyshp
# cf : http://gis.stackexchange.com/questions/183795/how-do-i-select-shapefiles-to-be-converted-to-geojson-in-folder-with-multiple-sh
# cf : https://github.com/mlaloux/Python-geo_interface-applications/blob/master/PyShp_geointerface.py

# cf : https://glenbambrick.com/tag/pyshp/
# cf : https://glenbambrick.com/2016/01/24/reproject-shapefile/

In [21]:
### SHP files 
_shp  = ".shp"
_json = ".json"
_copy = "_copy"

water_shp_fname = "PolygMasseDEauSouterraine"

## TO DO : add departements .shp 

In [22]:
### reading/writing - converting shp files to geojson
    
def geofile_path(filename, extension):
    path = os.path.join(carto_path , filename + extension )
    print path
    return path


In [23]:
'''
def readSHP(filename):  
    # generator 
    reader = shapefile.Reader( root_carto_folder+ filename + _shp )  
    fields = reader.fields[1:]  
    field_names = [field[0] for field in fields]  
    for sr in reader.shapeRecords():  
        geom = sr.shape.__geo_interface__  
        atr = dict(zip(field_names, sr.record))  
        yield dict(geometry=geom,properties=atr)    
'''

def readSHP(filename) :
    
    # read the shapefile
    reader = shapefile.Reader( geofile_path( water_shp_fname, _shp ) )
    fields = reader.fields[1:]
    field_names = [field[0] for field in fields]

    buffer = []
    for sr in reader.shapeRecords():
       atr = dict(zip(field_names, sr.record))
       geom = sr.shape.__geo_interface__
       buffer.append(dict(type="Feature", geometry=geom, properties=atr)) 

    # write the GeoJSON file (copy)
    geojson = open( os.path.join( carto_path, filename + _copy + _json), "w")
    geojson.write(dumps({"type": "FeatureCollection", "features": buffer}, indent=2) + "\n")
    geojson.close()
    

In [24]:
### options for gdf.read_file()
import fiona; help(fiona.open)

### options for gdf.to_file()
#import fiona; fiona.supported_drivers

Help on function open in module fiona:

open(path, mode='r', driver=None, schema=None, crs=None, encoding=None, layer=None, vfs=None, enabled_drivers=None, crs_wkt=None)
    Open file at ``path`` in ``mode`` "r" (read), "a" (append), or
    "w" (write) and return a ``Collection`` object.
    
    In write mode, a driver name such as "ESRI Shapefile" or "GPX" (see
    OGR docs or ``ogr2ogr --help`` on the command line) and a schema
    mapping such as:
    
      {'geometry': 'Point',
       'properties': [('class', 'int'), ('label', 'str'),
                      ('value', 'float')]}
    
    must be provided. If a particular ordering of properties ("fields"
    in GIS parlance) in the written file is desired, a list of (key,
    value) pairs as above or an ordered dict is required. If no ordering
    is needed, a standard dict will suffice.
    
    A coordinate reference system for collections in write mode can be
    defined by the ``crs`` parameter. It takes Proj4 style mappings lik

In [7]:
### read .shp with geopandas

gdf = geopandas.read_file( geofile_path( water_shp_fname, _shp ), crs=None, encoding=None )

gdf.crs

{u'ellps': u'GRS80',
 u'lat_0': 46.5,
 u'lat_1': 49,
 u'lat_2': 44,
 u'lon_0': 3,
 u'no_defs': True,
 u'proj': u'lcc',
 u'units': u'm',
 u'x_0': 700000,
 u'y_0': 6600000}

In [27]:
print gdf.columns.values
print
print gdf.loc[0]

gdf.head()

[u'CdBassinDC' u'CdEcoregio' u'CdEuMasseD' u'CdMasseDEa' u'CdPolygMas'
 u'Commentair' u'DateCreati' u'DateMajMas' u'FrangeLitt' u'Karstique'
 u'LatMasseDE' u'LonMasseDE' u'MasseDEauA' u'MasseDEauT' u'NatureEcou'
 u'Niveau' u'NomMasseDE' u'PrecSupMas' u'Regroupees' u'StMasseDEa'
 u'SurfaceAff' u'SurfaceSsC' u'SurfaceTot' u'SystemeRef' u'TypeMasseD'
 'geometry']

CdBassinDC                                                    G
CdEcoregio                                                   13
CdEuMasseD                                              FRGG094
CdMasseDEa                                                GG094
CdPolygMas                                             00001101
Commentair    Ancien libellÃ© de masse d'eau: Sables et argi...
DateCreati                                           31/12/2004
DateMajMas                                           18/12/2013
FrangeLitt                                                    N
Karstique                                                   

,CdBassinDC,CdEcoregio,CdEuMasseD,CdMasseDEa,CdPolygMas,Commentair,DateCreati,DateMajMas,FrangeLitt,Karstique,...,NomMasseDE,PrecSupMas,Regroupees,StMasseDEa,SurfaceAff,SurfaceSsC,SurfaceTot,SystemeRef,TypeMasseD,geometry
0,G,13,FRGG094,GG094,00001101,Ancien libellÃ© de masse d'eau: Sables et argi...,31/12/2004,18/12/2013,N,N,...,Sables et argiles miocÃ¨nes de Sologne libres,1,Y,ValidÃ©,3904.5,369.2,4273.7,26,IL,(POLYGON ((602460.1085679978 6690396.603922732...
1,G,13,FRGG136,GG136,00001094,Ancien libellÃ© de masse d'eau: Calcaires tert...,31/12/2004,18/12/2013,N,Y,...,Calcaires tertiaires de Beauce sous Sologne ca...,1,N,ValidÃ©,0.0,3248.8,3248.8,26,DS,(POLYGON ((602460.1085679978 6690396.603922732...
2,G,13,FRGG095,GG095,00001105,Ancien libellÃ© de masse d'eau: Sables et calc...,31/12/2004,18/12/2013,N,N,...,Sables et calcaires lacustres des bassins tert...,1,Y,ValidÃ©,1647.2,31.8,1679.0,26,IL,(POLYGON ((584337.9010915682 6701839.975092072...
3,G,13,FRGG089,GG089,00001106,Ancien libellÃ© de masse d'eau: Craie du SÃ©no...,31/12/2004,18/12/2013,N,N,...,Craie du SÃ©no-Turonien sous Beauce sous Solog...,1,N,ValidÃ©,50.9,4458.2,4509.1,26,DS,(POLYGON ((602460.1085679978 6690396.603922732...
4,H,13,FRHG210,HG210,00001100,None,31/12/2004,18/12/2013,N,Y,...,Craie du GÃ¢tinais,1,Y,ValidÃ©,3565.5,57.5,3623.0,26,DS,"POLYGON ((681077.108196184 6722325.508935641, ..."


In [18]:
### test read/write /// .shp --> .json

#readSHP(water_shp_fname) ### problem : 1.16 Go file !!!